In [2]:
from keras.models import load_model
import cv2
import warnings
import numpy as np
from matplotlib.patches import Polygon
import matplotlib.pyplot as plt
from PIL import Image
from PIL import ImageEnhance
from decimal import Decimal, ROUND_HALF_UP
import warnings
warnings.simplefilter('ignore', FutureWarning)

Using TensorFlow backend.


In [3]:
cancer_labels = {0:'nv', 1:'mel', 2:'bcc'}
model = load_model('nv_cancer_cnn_raw_female2.h5')

In [4]:
def resize(img):
    width, height = img.shape[1], img.shape[0]
    arr1 = img[((height-75)//2):((height+75)//2),((width-100)//2):((width+100)//2)]
    return arr1

def predict(img):
#     rp = np.zeros((75,100,3))
#     rp.T[0] += img.T[2]
#     rp.T[1] += img.T[1]
#     rp.T[2] += img.T[0]
    rp = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    im = (rp - np.mean(rp)) / np.std(rp)
    data = im.reshape((1,75,100,3))
    pred = model.predict(data)
    bestnum = 0.0
    bestclass = 0
    for n in [0,1,2]:
        if bestnum < pred[0][n]:
            bestnum = pred[0][n]
            bestclass = n
    return bestclass, pred

def saturation(img, rate):
    bf_img = ImageEnhance.Color(Image.fromarray(img))
    af_img = bf_img.enhance(rate)
    return np.array(af_img)

def brightness(img, rate):
    bf_img = ImageEnhance.Brightness(Image.fromarray(img))
    af_img = bf_img.enhance(rate)
    return np.array(af_img)

def contrast(img, rate):
    bf_img = ImageEnhance.Contrast(Image.fromarray(img))
    af_img = bf_img.enhance(rate)
    return np.array(af_img)

def sharpness(img, rate):
    bf_img = ImageEnhance.Sharpness(Image.fromarray(img))
    af_img = bf_img.enhance(rate)
    return np.array(af_img)

def roundnum(num):
    roundnum = Decimal(str(num)).quantize(Decimal('0.001'), rounding=ROUND_HALF_UP)
    return roundnum

In [5]:
def show_webcam(mirror=False):
    scale = 20

    cam = cv2.VideoCapture(0)
    while True:
        ret_val, image = cam.read()
        if mirror: 
            image = cv2.flip(image, 1)


        #get the webcam size
        height, width, channels = image.shape

        #prepare the crop
        centerX,centerY=int(height/2),int(width/2)
        radiusX,radiusY= int(scale*height/100),int(scale*width/100)
        
        minX,maxX=centerX-radiusX,centerX+radiusX
        minY,maxY=centerY-radiusY,centerY+radiusY

        cropped = image[minX:maxX, minY:maxY]
        resized_cropped = cv2.resize(cropped, (width, height))
        resize_img = resize(resized_cropped)         #resize img_array -> (75,100,3)
        
        ## Image processing 
        # down saturation
        sat_img = saturation(resize_img, 0.8)
        # up brightness
        brt_img = brightness(sat_img, 1.2)
        # up contrast
        cst_img = contrast(brt_img, 0.8)
        # up sharpness
        shp_img = sharpness(cst_img, 2.0)
        
        ## create 'resize' view
        # mask the target
        cst = shp_img.copy()
        imgray = cv2.cvtColor(cst,cv2.COLOR_BGR2GRAY)
        ret,thresh = cv2.threshold(imgray,127,255,0)
        contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

        if len(contours) > 1:
            cv_img = cv2.drawContours(cst, contours, -1, (150,255,150), 3)
            cv2.imshow('resize', cst)
        else:
            cv2.imshow('resize', shp_img)
        
        ## create 'prediction' view
        # prediction
        num, pred = predict(shp_img)
        nv, mel, bcc = roundnum(pred[0][0]), roundnum(pred[0][1]), roundnum(pred[0][2])
        cv2.putText(resized_cropped, cancer_labels[num], (0,50), cv2.FONT_HERSHEY_PLAIN, 4, (83, 46, 230), 3, cv2.LINE_AA)
        cv2.putText(resized_cropped, f'nv:{nv}', (0,90), cv2.FONT_HERSHEY_PLAIN, 2, (193, 230, 46), 3, cv2.LINE_AA)
        cv2.putText(resized_cropped, f'mel:{mel}', (0,120), cv2.FONT_HERSHEY_PLAIN, 2, (193, 230, 46), 3, cv2.LINE_AA)
        cv2.putText(resized_cropped, f'bcc:{bcc}', (0,150), cv2.FONT_HERSHEY_PLAIN, 2, (193, 230, 46), 3, cv2.LINE_AA)            
        cv2.imshow('prediction', resized_cropped)
        
        ## key operation
        k = cv2.waitKey(1)
        if k == 27: 
            break  # esc to quit
            
        # save a pic
        elif k == ord('s'):
            cv2.imwrite('scr2.jpg', shp_img)
        
        # add + or - 5 % to zoom
        elif k == ord('0'): 
            scale += 2  # +2

        elif k == ord('1'): 
            scale -= 2  # +2

    cv2.destroyAllWindows()


def main():
    show_webcam(mirror=True)


if __name__ == '__main__':
    main()